In [7]:
from __future__ import print_function 
import os, sys, h5py
import numpy as np
from six.moves import cPickle
from collections import OrderedDict

import tensorflow as tf
sys.path.append('..')
from tfomics import neuralnetwork_dev as nn
from tfomics import neuralbuild as nb
from tfomics import utils, learn, init

# import models
#from model_zoo import test_model

In [8]:
filename = 'processed_dataset.hdf5'
#data_path = 'D:/Dropbox/TFconnect'
data_path = '/Users/juliankimura/Desktop/tensorflow/data'
filepath = os.path.join(data_path,filename)

group_name = ['processed_data']
dataset = h5py.File(filepath,'r')
%time dtf = np.array(dataset['/'+group_name[0]+'/dtf'])
ltf = np.array(dataset['/'+group_name[0]+'/ltf'])
dtf_crossval = np.array(dataset['/'+group_name[0]+'/dtf_crossval'])
ltf_crossval = np.array(dataset['/'+group_name[0]+'/ltf_crossval'])

X_train = dtf[:80000,:,:,:]
y_train = ltf[:80000,:]
X_valid = dtf_crossval[:1000,:,:,:]
y_valid = ltf_crossval[:1000,:]

CPU times: user 991 ms, sys: 3.21 s, total: 4.2 s
Wall time: 4.35 s


In [3]:

def model(input_shape, num_labels=None):

    # create model
    layer1 = {'layer': 'input',
            'input_shape': input_shape,
            'name': 'input'
            }
    layer2 = {'layer': 'conv2d', 
            'num_filters': 16,
            'filter_size': (2, 5),
            'norm': 'batch',
            'activation': 'relu',
            'dropout': 0.1,
            'name': 'conv1'
            }
    layer4 = {'layer': 'conv2d', 
            'num_filters': 32,
            'filter_size': (2, 5),
            'norm': 'batch',
            'activation': 'relu',
            'dropout': 0.1,
            'name': 'conv2'
            }
    layer6 = {'layer': 'conv2d', 
            'num_filters': 10,
            'filter_size': (1,1),
            'norm': 'batch',
            'activation': 'relu',
            'dropout': 0.1,
            'name': 'conv3'
            }
    layer7 = {'layer': 'dense', 
            'num_units': 64,
            'norm': 'batch',
            'activation': 'relu',
            'dropout': 0.2,
            'name': 'dense1'
            }  
    layer9 = {'layer': 'dense', 
            'num_units': num_labels,
            'activation': 'softmax',
            'name': 'dense2'
            }

    #from tfomics import build_network
    model_layers = [layer1, layer2, layer4, layer6, layer7, layer9]

    # optimization parameters
    optimization = {"objective": "categorical",
                  "optimizer": "adam",
                  "learning_rate": 0.001,      
                  "l2": 1e-6,
                  # "l1": 0, 
                  }
    return model_layers, optimization
# get shapes
num_data, height, width, dim = X_train.shape
input_shape=[None, height, width, dim]
num_labels = y_train.shape[1]  
model_layers, optimization = model(input_shape, num_labels)
nnbuild = nb.NeuralBuild(model_layers)
network, placeholders, hidden_feed_dict = nnbuild.get_network_build()

In [ ]:

def model(input_shape, num_labels=None):
    # design a neural network model

    inputs = utils.placeholder(shape=input_shape, name='input')
    is_training = tf.placeholder(tf.bool, name='is_training')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    targets = utils.placeholder(shape=(None,num_labels), name='output')

    # placeholder dictionary
    placeholders = {'inputs': inputs, 
                  'targets': targets, 
                  'keep_prob': keep_prob, 
                  'is_training': is_training}


    # create model
    layer1 = {'layer': 'input',
                        'inputs': inputs,
                        'name': 'input'
                        }
    layer2 = {'layer': 'conv2d', 
                        'num_filters': 25,
                        'filter_size': (19,1),
                        'W': init.GlorotUniform(),
                        'b': init.Constant(0.1),
                        #'batch_norm': is_training,
                        'padding': 'SAME',
                        'activation': 'relu',
                        'pool_size': (40,1),
                        'name': 'conv1'
                        }
    layer3 = {'layer': 'residual-conv2d',
                        'filter_size': (5,1),
                        'batch_norm': is_training,
                        'dropout': keep_prob,
                        'pool_size': (40,1),
                        'name': 'resid1'
                     }
    layer4 = {'layer': 'dense', 
                'num_units': 128,
                'activation': 'relu',
                'W': init.GlorotUniform(),
                'b': init.Constant(0.1),
                'dropout': keep_prob,
                'name': 'dense1'
                }
    layer5 = {'layer': 'dense', 
                'num_units': num_labels,
                'W': init.GlorotUniform(),
                'b': init.Constant(0.1),
                'activation': 'sigmoid',
                'name': 'dense2'
                }

    #from tfomics import build_network
    model_layers = [layer1, layer2, layer4, layer5]
    net = build_network(model_layers)

    # optimization parameters
    optimization = {"objective": "binary",
                    "optimizer": "adam",
                    "learning_rate": 0.001,      
                    "l2": 1e-6,
                    # "l1": 0, 
                    }

    return net, placeholders, optimization

# get shapes
num_data, height, width, dim = X_train.shape
input_shape=[None, height, width, dim]
num_labels = y_train.shape[1]  
model_layers, optimization = model(input_shape, num_labels)



In [5]:

# build neural network class
nnmodel = nn.NeuralNet(network, placeholders, hidden_feed_dict)
nnmodel.inspect_layers()

# set output file paths

results_path = utils.make_directory(data_path, 'results')
results_path = utils.make_directory(results_path, 'tfomics')
output_name = 'test'
filepath = os.path.join(results_path, output_name)

# compile neural trainer
nntrainer = nn.NeuralTrainer(nnmodel, optimization, save='best', filepath=filepath)

----------------------------------------------------------------------------
Network architecture:
----------------------------------------------------------------------------
layer1: input
(?, 3, 330, 1)
layer2: conv1
(?, 2, 326, 16)
layer3: conv1_batch
(?, 2, 326, 16)
layer4: conv1_active
(?, 2, 326, 16)
layer5: conv1_dropout
(?, 2, 326, 16)
layer6: conv2
(?, 1, 322, 32)
layer7: conv2_batch
(?, 1, 322, 32)
layer8: conv2_active
(?, 1, 322, 32)
layer9: conv2_dropout
(?, 1, 322, 32)
layer10: conv3
(?, 1, 322, 10)
layer11: conv3_batch
(?, 1, 322, 10)
layer12: conv3_active
(?, 1, 322, 10)
layer13: conv3_dropout
(?, 1, 322, 10)
layer14: dense1
(?, 64)
layer15: dense1_bias
(?, 64)
layer16: dense1_batch
(?, 64)
layer17: dense1_active
(?, 64)
layer18: dense1_dropout
(?, 64)
layer19: dense2
(?, 2)
layer20: dense2_bias
(?, 2)
layer21: output
(?, 2)
----------------------------------------------------------------------------


In [6]:
train = {'inputs': [X_train], 'targets': y_train}
valid = {'inputs': [X_valid], 'targets': y_valid}
data = {'train': train, 'valid': valid}
learn.train_minibatch(nntrainer, data, batch_size=200, num_epochs=50, 
                    patience=10, verbose=2, shuffle=True)

Epoch 1 out of 50 
[=====================         ] 70.5% -- time=32s -- loss=0.92538 -- accuracy=50.56%  

KeyboardInterrupt: 

In [ ]:

batch_mean, batch_var = tf.nn.moments(, [0])

In [ ]:
x = tf.identity([1., 2.])

In [ ]:
ema_op = ema.apply(x)

In [ ]:
sess = tf.Session()
sess.run(ema)